In [1]:
import re
import json
import tqdm
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [20]:
df = pd.read_csv("../data/sentiment/train.csv")
df.drop_duplicates("text", inplace=True)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df

,text,start_char_pos,end_char_pos,important_span_text,sentiment
0,Epic Games has raised $2 billion in funding fr...,11,43,has raised $2 billion in funding,positive
1,The Halt is the best thing that's happened to ...,16,42,best thing that's happened,positive
2,#HelloFuture would just be augmented to #Hello...,13,41,would just be augmented to #,positive
3,A show of confidence that HBAR can stand on it...,31,63,can stand on its own as an asset,negative
4,": we want it, you need it!",0,13,": we want it,",positive
...,...,...,...,...,...
9992,The team promises that the project enables fas...,35,72,"enables fast, near-zero cost payments",positive
9993,The launchpad represents an IDO platform for i...,14,72,represents an IDO platform for issuing new coi...,positive
9994,The system's marketplace allows users to produ...,25,60,"allows users to produce, mint, sell",positive
9995,Ezillion is a multi-utility token that will us...,3,65,Ezillion is a multi-utility token that will us...,positive


In [42]:
from transformers import pipeline
pipeline = pipeline('feature-extraction', model="roberta-base", return_tensors="tf")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
len(pipeline("this is a test")[0][0])

768

In [41]:
data = []
labels = []

for row in df.itertuples():
    encoded = tokenizer(row.text)
    tokens = encoded["input_ids"]

    l = 0
    res = ""
    start = row.text.index(row.important_span_text)
    end = start + len(row.important_span_text)
    decoded = []
    
    y = [0]
    p = 1
    
    assert row.important_span_text in row.text
    
    for tok in tokens[1:-1]:
        dec = tokenizer.decode(tok)
        decoded.append(dec)

        if start - 1 <= l < end:
            res = res + dec
            y.append(p)
        else:
            y.append(0)
        
        l += len(dec)

    res = res[1:] if res.startswith(" ") else res
    y.append(0)
    vectors = pipeline(row.text)[0]
    
    assert l == len(row.text), decoded
    assert res == row.important_span_text
    assert len(vectors) == len(y)
    
    
    for i in range(1, len(y)):
        data.append(vectors[i-1:i+2])
        labels.append(y[i])

KeyboardInterrupt: 

In [71]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, "x", data)


In [90]:
results = []

for row in tqdm.tqdm(data, total=len(data)):
    all_options = []
    res = ""
    text = row["text"]
    text = text.replace("’", "'").replace("“", "\"").replace("”", "\"").replace("–","-").replace("…", ".").replace("‘", "'").replace("—", "-")
    text = remove_emojis(text)

    for word in nlp(text):
        if word["entity"] != "LABEL_0":
            res = res + word["word"].replace("Ġ", " ")
        else:
            all_options.append(res)
            res = ""

    all_options.append(res)
    sel = max(all_options, key=lambda x: len(x))
    cleaned = re.sub("(^[\:, ]+)|([\. ]+$)", "", sel)
    start = 0
    end = 0
    if len(cleaned) > 1:
        if cleaned in text:
            start = text.index(cleaned)
            end = start + len(cleaned)
        #else:
        #    end = cleaned
    results.append((start, end))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [22:21<00:00,  7.46it/s]


In [91]:
submission = pd.DataFrame(data=results, columns=["start", "end"])

In [95]:
submission.reset_index().rename(columns={"index": "id"}).to_csv("../predictions/t2_roberta_tokenizer.csv",index=False)